In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import numpy as np
from pyuvdata import UVCal, UVData, utils
import os
import sys
import glob
import uvtools as uvt
from astropy.time import Time
from astropy.coordinates import EarthLocation, SkyCoord, AltAz, Angle
import pandas
import warnings 
import copy
import utils
from hera_mc import cm_hookup
warnings.filterwarnings('ignore')

In [ ]:
# get data location
data_path = os.environ['DATA_PATH']

In [ ]:
# Load in data
HHfiles, difffiles, uvdx, uvdy, uvd1, uvd2 = utils.load_data(data_path)

## Antenna Positions

In [ ]:
utils.plot_antenna_positions(uvd_sum)

## Autocorrelations for a single file

This plot shows autocorrelations for one timestamp of each antenna that is active and each polarization

In [ ]:
### plot autos
utils.plot_autos(uvdx, uvdy, uvd1, uvd2)

## Waterfalls of Autocorrelation Amplitudes for each Antenna and Each polarization

In [ ]:
uvd = UVData()
uvd.read_uvh5(HHfiles, ant_str = 'autos')
lst0 = np.min(uvd1.lst_array) 
lst1 = np.max(uvd2.lst_array)

In [ ]:
utils.plot_wfs(uvd, pol = 0)

In [ ]:
utils.plot_wfs(uvd, pol = 1)

## Correlation Matrices

Shows the phase correlation between antennas. Using the even and odd visibilities, each pixel is calculated as (even/abs(even)) * (conj(odd)/abs(odd)), and then averaged across time and frequency. If the phases are noise-like, this value will average down to zero. If the antennas are well correlated, the phases should not be noise-like, and this value should average to 1. The lines denoting node boundaries are intended to help confirm that inter-node correlations are functioning - if they aren't, this plot will appear block-diagonal.

Note: Within each node, the order of antennas is determined by snap, and within that by snap input. 

In [ ]:
sm0,sm1,sm2,df0,df1,df2 = utils.load_sum_and_diff_files(HHfiles, difffiles)
matrix0 = utils.calcEvenOddAmpMatrix(sm0,df0,nodes='auto')
matrix1 = utils.calcEvenOddAmpMatrix(sm1,df1,nodes='auto')
matrix2 = utils.calcEvenOddAmpMatrix(sm2,df2,nodes='auto')
utils.plotCorrMatrix(sm0, matrix0, nodes='auto')
utils.plotCorrMatrix(sm1, matrix1, nodes='auto')
utils.plotCorrMatrix(sm2, matrix2, nodes='auto')